In [35]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [14]:
df=pd.read_excel("./data/Data_Novartis_Datathon-Participants.xlsx",skiprows=3)

In [15]:
df.head(10)

,Cluster,Brand Group,Country,Function,Jan 2012,Feb 2012,Mar 2012,Unnamed: 7,Apr 2012,May 2012,...,Unnamed: 113,Jul 2018,Aug 2018,Sep 2018,Unnamed: 117,Oct 2018,Nov 2018,Dec 2018,Unnamed: 121,Unnamed: 122
0,Cluster 3,Brand Group 12,Country 19,Sales 1,1861.328751,1995.945017,1998.533792,5855.807560,2019.243986,2425.681558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cluster 3,Brand Group 12,Country 19,Sales 2,1316.247027,1450.223283,1440.837462,4207.307772,1564.066421,2014.132904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cluster 3,Brand Group 12,Country 19,Investment 1,-216.162658,-238.167239,-217.457045,-671.786942,-214.868270,-390.904926,...,-431.807560,-146.395189,-145.100802,-171.894616,-463.390607,-136.946163,-189.974966,-156.680908,-483.602037,-1816.303068
3,Cluster 3,Brand Group 12,Country 19,Investment 2,-107.434135,-14.238259,-62.130584,-183.802978,-91.901489,-199.335624,...,-155.455899,-35.595647,-64.460481,-111.705613,-211.761741,-29.253150,-133.434491,-88.018327,-250.705968,-696.492909
4,Cluster 3,Brand Group 12,Country 19,Investment 4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-3.106529,-3.106529,0.000000,0.000000,0.000000,0.000000,-1.423826
5,Cluster 3,Brand Group 12,Country 19,Investment 6,-56.953036,-55.658648,-64.719359,-177.331042,-56.953036,-55.658648,...,-149.890034,-42.197022,-41.032073,-48.539519,-131.768614,-45.044674,-45.007720,-35.730998,-125.783392,-544.129325
6,Cluster 3,Brand Group 17,Country 19,Sales 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cluster 3,Brand Group 17,Country 19,Sales 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Cluster 3,Brand Group 17,Country 19,Investment 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-6300.688431,-2123.312715,-1912.457045,-2342.970218,-6378.739977,-2114.899198,-2206.086863,-1968.091008,-6289.077068,-24996.207653
9,Cluster 3,Brand Group 17,Country 19,Investment 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-7063.470791,-1790.266896,-1048.194731,-1904.690722,-4743.152348,-1511.585338,-1595.179450,-1041.791168,-4148.555956,-19825.655613


##  Sales1

In [16]:
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
dfsales2=df.loc[df['Function'] == "Sales 2"]
dfsales2 = dfsales2[dfsales2.columns.drop(list(dfsales2.filter(regex='2018')))]
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')
dfmelt=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value_Y")

In [17]:
dfmelt.head(5)

,Cluster,Brand Group,Date,Value_Y
0,Cluster 1,Brand Group 12,Jan 2012,616.408961
1,Cluster 1,Brand Group 13,Jan 2012,62.381470
2,Cluster 1,Brand Group 15,Jan 2012,0.000000
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992
4,Cluster 1,Brand Group 17,Jan 2012,0.000000


## Sales 2

In [31]:
def add_lag_values(dfpre,lag,lagname):
    for i in range (len (dfpre)):
        if dfpre.iloc[i].Date_coded>12:
            dfpre.iloc[i,dfpre.columns.get_loc(lagname)]=dfpre[(dfpre["Cluster"]==dfpre.iloc[i].Cluster) 
                   & (dfpre["Brand Group"]==dfpre.iloc[i,dfpre.columns.get_loc("Brand Group")])
                                                                &(dfpre["Date_coded"]==dfpre.iloc[i].Date_coded-lag)]["Value_Y"].values[0]
    return dfpre

In [36]:
dictmonths=dict(zip(dfmelt["Date"].unique(),range(len(dfmelt["Date"].unique()))))
dfmelt["Date_coded"]=dfmelt["Date"].apply(lambda x: dictmonths[x])
dfmelt["Month"]=dfmelt["Date_coded"].apply(lambda x: x%12)
dfmelt["Actual"] = np.nan
dfmelt["Lag-1"] = np.nan
dfmelt["Lag-2"] = np.nan
dfmelt["Lag-3"] = np.nan

In [ ]:
dfpre=add_lag_values(dfmelt,1,'Actual')
dfpre=add_lag_values(dfmelt,2,'Lag-1')
dfpre=add_lag_values(dfpre,3,'Lag-2')
dfpre=add_lag_values(dfpre,4,'Lag-3')

In [ ]:
dfpre['Month']=dfpre['Month'].apply(lambda x: 11 if x == 0 else x-1)

In [ ]:
dfpre['sales_trend'] = dfpre['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpre['sales_trend'] -= dfpre[ft_name]

dfpre['sales_trend'] /= len(lag_list) + 1

In [ ]:
dfpre.head(5)

In [ ]:
dfpre_sales['Date_coded']=dfpre_sales['Date_coded'].apply(lambda x: x-1)

In [ ]:
dfinves.head(5)

In [ ]:
dffinal=pd.merge(dfpre_sales,dfinves,
                 on=['Cluster','Brand Group','Date_coded'])

In [ ]:
dffinal.head(10)

In [3]:
dffinal=pd.read_pickle("./data/lags_invest.pkl")

In [4]:
yvalues =dffinal["Value_Y"]
xvalues = dffinal.drop(labels=['Value_Y'],axis=1)

In [710]:
from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
##sc.fit(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])
#xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(xvalues, yvalues, test_size=0.2, shuffle=True)

In [712]:
xvalues.columns

Index(['Cluster', 'Brand Group', 'Date', 'Date_coded', 'Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'sales_trend', 'Investment 1',
       'Investment 2', 'Investment 3', 'Investment 4', 'Investment 5',
       'Investment 6'],
      dtype='object')

In [7]:
X_train=X_train[['Month','Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']]
X_valid=X_valid[['Month','Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']]

In [8]:
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'xgboost'

In [9]:
model = XGBRegressor(
    max_depth=5,
    n_estimators=1000,
    min_child_weight=10, 
    colsample_bytree=0.5, 
    subsample=0.9, 
    eta=0.75,    
    seed=42)

model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    verbose=True, 
    early_stopping_rounds = 30)

NameError: name 'XGBRegressor' is not defined

# Pred dataset

In [18]:
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')

In [19]:
dfpred=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value")

In [20]:
dfindexes=dfpred=dfsales2.reset_index()[["Cluster","Brand Group"]]

In [21]:
# GET LABELSSSSSSSSSSSSSSSs

In [22]:
dfpred=dfsales2.reset_index()[["Sep 2017","Oct 2017","Nov 2017","Dec 2017"]]

In [25]:
dfpred["Month"]=11

In [27]:
dfpred.columns=["Lag-3","Lag-2","Lag-1","Actual","Month"]

In [28]:
dfpred['sales_trend'] = dfpred['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpre['sales_trend'] -= dfpre[ft_name]

dfpre['sales_trend'] /= len(lag_list) + 1

NameError: name 'dfpre' is not defined

In [662]:
dfpred.head(5)

,Lag-3,Lag-2,Lag-1,Actual,Month,sales_trend
0,1608.715411,1538.061964,1574.399619,1381.174249,12,1381.174249
1,5.803457,5.608148,8.537778,6.361481,12,6.361481
2,35.634313,177.382970,48.785139,-265.362597,12,-265.362597
3,612.722539,535.401404,566.573007,539.184516,12,539.184516
4,2283.761079,2830.252855,2530.420686,2870.710849,12,2870.710849


In [663]:
#dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [664]:
def add_values(pred_x, new_pred,month):
    pred_x['Month']=month
    
    pred_x["Lag-3"]=pred_x["Lag-2"]
    pred_x["Lag-2"]=pred_x["Lag-1"]
    pred_x["Lag-1"]=pred_x["Actual"]
    pred_x["Actual"]=new_pred
    pred_x['sales_trend'] = pred_x['Actual']
    lag_list=[1,2,3]

    for lag in [1,2,3]:
        ft_name = ('Lag-%s' % lag)
        pred_x['sales_trend'] -= pred_x[ft_name]

    pred_x['sales_trend'] /= len(lag_list) + 1
    
    return pred_x

In [665]:
list_predictions=[]

In [666]:
xpred=dfpred[['Month', 'Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']]

In [667]:
for i in range(12):
    preds=model.predict(xpred)
    list_predictions.append(preds)
    xpred=add_values(xpred, preds,i)

In [668]:
len(list_predictions[0])

303

In [669]:
for d in range(len(list_predictions)):
    dfindexes['Month-%s' % d]=list_predictions[d]

In [670]:
dfindexes["Brand Group"].unique()

array(['Brand Group 12', 'Brand Group 13', 'Brand Group 15',
       'Brand Group 16', 'Brand Group 17', 'Brand Group 18',
       'Brand Group 19', 'Brand Group 24', 'Brand Group 25',
       'Brand Group 26', 'Brand Group 27', 'Brand Group 28',
       'Brand Group 30', 'Brand Group 31', 'Brand Group 36',
       'Brand Group 38', 'Brand Group 39', 'Brand Group 40',
       'Brand Group 41', 'Brand Group 43', 'Brand Group 44',
       'Brand Group 45', 'Brand Group 46', 'Brand Group 51',
       'Brand Group 58', 'Brand Group 63', 'Brand Group 70',
       'Brand Group 71', 'Brand Group 72', 'Brand Group 73',
       'Brand Group 76', 'Brand Group 78', 'Brand Group 79',
       'Brand Group 80', 'Brand Group 84', 'Brand Group 85',
       'Brand Group 87', 'Brand Group 89', 'Brand Group 90',
       'Brand Group 92', 'Brand Group 94', 'Brand Group 96',
       'Brand Group 97', 'Brand Group 20', 'Brand Group 98',
       'Brand Group 60', 'Brand Group 64', 'Brand Group 21'], dtype=object)

In [671]:
dfindexes

,Cluster,Brand Group,Month-0,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11
0,Cluster 1,Brand Group 12,1632.197632,1211.584351,1377.070679,1342.979492,1334.370850,1314.214111,1318.171875,1292.714355,1344.660767,1330.777466,1337.252075,1358.666138
1,Cluster 1,Brand Group 13,81.828377,5.216658,7.284895,7.227825,9.626293,6.860891,6.082820,5.639297,5.837836,3.032424,7.160820,9.652314
2,Cluster 1,Brand Group 15,-17.568844,-152.548080,-160.863312,-225.174866,-225.174866,-225.796204,-225.796204,-236.823608,-236.823608,-219.102570,-207.074249,-208.657043
3,Cluster 1,Brand Group 16,716.253052,497.814423,532.475098,572.096741,580.056580,582.576111,586.883728,505.316315,540.771851,578.422729,589.919495,580.464600
4,Cluster 1,Brand Group 17,2718.486084,2671.920166,2842.251221,2815.820068,2832.056885,2805.912842,2999.287842,2625.524414,2763.027832,2768.531006,2758.083740,2734.190674
5,Cluster 1,Brand Group 18,477.937714,200.379440,330.634918,323.064392,329.316345,334.898804,343.011444,286.582672,310.745758,319.004303,318.330994,329.414490
6,Cluster 1,Brand Group 19,1379.575562,745.721375,692.558777,660.516602,665.254578,671.182861,677.866089,664.225708,673.617432,683.509705,683.509705,676.470154
7,Cluster 1,Brand Group 24,1553.715942,1191.722290,1359.901611,1384.113281,1387.187500,1377.210449,1345.177979,1272.497070,1381.827881,1383.881226,1372.453857,1370.119263
8,Cluster 1,Brand Group 25,1569.044922,1171.419434,1240.051880,1177.973389,1181.991577,1181.991577,1208.736328,1113.591553,1216.955933,1202.696411,1192.226440,1227.549927
9,Cluster 1,Brand Group 26,2598.563721,2372.460449,2551.495361,2505.745850,2470.091064,2481.990234,2621.901367,2249.249268,2370.468750,2438.719482,2453.526123,2476.946045


In [672]:
dfindexes.iloc[0][2:]

Month-0      1632.2
Month-1     1211.58
Month-2     1377.07
Month-3     1342.98
Month-4     1334.37
Month-5     1314.21
Month-6     1318.17
Month-7     1292.71
Month-8     1344.66
Month-9     1330.78
Month-10    1337.25
Month-11    1358.67
Name: 0, dtype: object

In [673]:
dir_submission_template = 'Data_Novartis_Datathon-Results_Challenge1_Template.csv'

submission_template = pd.read_csv(dir_submission_template)

bg_set = set()

#a = pd.DataFrame(final_res, columns=['Cluster', 'Brand Group', 'res'])

for i, r in submission_template.iterrows():
    bg = r['Brand Group'][12:].replace(',', '').split(' ')
    bg_set.update(bg)
    bg = [ 'Brand Group ' + str(i) for i in bg ]
    print(r['Cluster'], r['Brand Group'], bg)
    
#    print(type(r['res']))
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'] in bg ]
#    res = res['res']
    res = np.sum(res, axis=0)
    submission_template.iloc[i, 2:] = res
    
    
for i, r in submission_template[submission_template['Brand Group'] == 'others'].iterrows():
    print(r.Cluster)
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res1 = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'][12:] not in bg_set ]

    res2 = np.sum(res1, axis=0)
    submission_template.iloc[i, 2:] = res2

Cluster 1 Brand Group 17 ['Brand Group 17']
Cluster 2 Brand Group 17 ['Brand Group 17']
Cluster 3 Brand Group 17 ['Brand Group 17']
Cluster 4 Brand Group 17 ['Brand Group 17']
Cluster 5 Brand Group 17 ['Brand Group 17']
Cluster 8 Brand Group 17 ['Brand Group 17']
Cluster 9 Brand Group 17 ['Brand Group 17']
Cluster 10 Brand Group 17 ['Brand Group 17']
Cluster 1 Brand Group 24 ['Brand Group 24']
Cluster 3 Brand Group 24 ['Brand Group 24']
Cluster 4 Brand Group 24 ['Brand Group 24']
Cluster 5 Brand Group 24 ['Brand Group 24']
Cluster 8 Brand Group 24 ['Brand Group 24']
Cluster 9 Brand Group 24 ['Brand Group 24']
Cluster 10 Brand Group 24 ['Brand Group 24']
Cluster 1 Brand Group 30 ['Brand Group 30']
Cluster 2 Brand Group 30 ['Brand Group 30']
Cluster 4 Brand Group 30 ['Brand Group 30']
Cluster 5 Brand Group 30 ['Brand Group 30']
Cluster 7 Brand Group 30 ['Brand Group 30']
Cluster 8 Brand Group 30 ['Brand Group 30']
Cluster 9 Brand Group 30 ['Brand Group 30']
Cluster 10 Brand Group 30 ['Br

In [675]:
submission_template.to_csv("submission_2.csv",index=False)